In [1]:
# API KEY Loading
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("CH02-Runnable")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH02-Runnable


# RunnableParallel

- 시퀀스 내에서 
- 하나의 Runnabel의 출력을 다음 Runnable 입력 형식에 맞게 조작하는데 유용

In [15]:
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser

In [6]:
# Vector Store 생성
vector_store = FAISS.from_texts(
    [
        "영희는 생성형AI에 관심이 많은 AI 엔지니어 입니다."
    ],
    embedding=OpenAIEmbeddings(),
)

# Vector Store를 검색기로 사용
retriever = vector_store.as_retriever()

# 템플릿 정의
template = """
Answer the question based only on the following context: {context}
Question: {question}
"""

# Prompt 정의
prompt = PromptTemplate.from_template(template)

# 모델 정의/초기화
model = ChatOpenAI(model_name="gpt-4o-mini")

# Chain 구성
retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [7]:
retrieval_chain.invoke("영희의 직업은?")

'영희의 직업은 AI 엔지니어입니다.'

# itemgetter 사용하기

[참고] itemgetter : https://docs.python.org/3/library/operator.html#operator.itemgetter

In [8]:
from operator import itemgetter

In [9]:
# Vector Store 생성
vector_store = FAISS.from_texts(
    [
        "영희는 생성형AI에 관심이 많은 AI 엔지니어 입니다."
    ],
    embedding=OpenAIEmbeddings(),
)

# Vector Store를 검색기로 사용
retriever = vector_store.as_retriever()

# 템플릿 정의
template = """
Answer the question based only on the following context: {context}
Question: {question}
Answer in the following language: {language}
"""

# Prompt 정의
prompt = PromptTemplate.from_template(template)

# 모델 정의/초기화
model = ChatOpenAI(model_name="gpt-4o-mini")

# Chain 구성
chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [10]:
chain.invoke(
    {
        "question": "영희의 직업은?",
        "language": "English"
    }
)

"Younghee's profession is an AI engineer."

## 여러 체인을 묶어서 처리

In [21]:
language_chain = (
    PromptTemplate.from_template("{country}의 언어는 무엇입니까?")
    | model
    | StrOutputParser()
)

capital_chain = (
    PromptTemplate.from_template("{country}의 수도는 어디입니까?")
    | model
    | StrOutputParser()
)

map_chain = RunnableParallel(
    language=language_chain,
    capital=capital_chain
)

In [22]:
map_chain.invoke({"country":"포르투칼"})

{'language': '포르투갈의 공식 언어는 포르투갈어입니다. 포르투갈어는 전 세계적으로 많은 나라에서 사용되며, 특히 브라질, 앙골라, 모잠비크, 카보베르데, 기니비사우, 상투메프린시페 등에서 공용어로 사용됩니다.',
 'capital': '포르투갈의 수도는 리스본(Lisboa)입니다.'}

체인별로 각각 다른 변수를 받아서 병렬처리를 하면...

In [25]:
language_chain2 = (
    PromptTemplate.from_template("{country1}의 언어는 무엇입니까?")
    | model
    | StrOutputParser()
)

capital_chain2 = (
    PromptTemplate.from_template("{country2}의 수도는 어디입니까?")
    | model
    | StrOutputParser()
)

map_chain2 = RunnableParallel(
    language=language_chain2,
    capital=capital_chain2
)

In [26]:
map_chain2.invoke(
    {
        "country1":"포르투칼",
        "country2":"스페인",
    }
)

{'language': '포르투갈의 공식 언어는 포르투갈어입니다. 포르투갈어는 브라질, 앙골라, 모잠비크 등 여러 나라에서도 사용되며, 세계에서 네 번째로 가장 많이 사용되는 언어 중 하나입니다.',
 'capital': '스페인의 수도는 마드리드(Madrid)입니다.'}

# 병렬 처리 확인

단독으로 처리 되었을 때의 실행 시간과 
병렬 프로세스로 실행했을 때의 시간은 거의 차이 없음

In [27]:
%%timeit

language_chain.invoke({"country":"포르투칼"})

1.14 s ± 272 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
%%timeit

capital_chain.invoke({"country":"포르투칼"})

695 ms ± 88.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
%%timeit

map_chain.invoke({"country":"포르투칼"})

1.34 s ± 159 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


-----
* End of Document *